In [1]:
import pandas as pd
import numpy as np

In [8]:
def fred_transform(data: pd.DataFrame, transformations: dict[str, int]) -> pd.DataFrame:
    """ 
    Transforme les séries de données macro-économiques selon les recommandations fournies par McCracken et Ng (2016).

    Arguments
    ---------
    data : pd.DataFrame
        Tableau de données macro-économiques extraties de la base fred-md.
     transformations : dict[str, int]
        Dictionnaire ayant pour clé le nom de la série et pour valeur son code de transformation associé.
    """
    # Création du DataFrame stockant les séries transformées
    result = pd.DataFrame()
    # Transformation des séries selon les codes fournis par McCracken et Ng (2016)
    for col in data.columns:
        # Récupération du code de transformation
        code = transformations[col]
        # Récupération de la série individuelle
        subset = data[col]

        # Application des transformées selon le code correspondant
        # --------------------------------------------------------
        if code == 1:
            temp = subset # Aucune transformation à appliquer
        elif code == 2:
            temp = subset.diff(periods=1) # Différence première
        elif code == 3: 
            temp = subset.diff(periods=1).diff(periods=1) # Différence seconde
        elif code == 4:
            temp = np.log(subset) # Transformée en log
        elif code == 5:
            temp = np.log(subset).diff(periods=1) # Différence première du log
        elif code == 6:
            temp = np.log(subset).diff(periods=1).diff(periods=1) # Différence seconde du log 
        elif code == 7:
            temp = subset.pct_change().diff(periods=1) # Différence première de la variation relative
        # --------------------------------------------------------

        # Aggrégation des séries transformées
        result = pd.concat([result, temp], axis=1)
    # Renvoi des séries transformées et abandonnées des NaN's
    return result.dropna(axis=0)

def remove_outliers(transformed_data: pd.DataFrame) -> pd.DataFrame:
    """ 
    Traite les séries transformées de la base fred-md de leurs outliers.
    Un point x est considéré comme outlier si abs(x - mediane) > 10 * gamme interquartile.

    Arguments
    ---------
    transformed_data : pd.DataFrame
        Tableau des données macro-économiques transformées de la base fred-md
    """
    # Calcul de la médiane des séries
    medians = transformed_data.median(axis=0)
    # DataFrame contenant les médianes des séries en chaque point
    mdf =  transformed_data * 0 + medians
    # Calcul de la distance entre les observations et les médianes
    z = abs(transformed_data - mdf)
    # Calcul de la gamme interquartile des séries
    irq = transformed_data.quantile(q=.75) - transformed_data.quantile(q=.25)
    # DataFrame contenant les gammes interquartiles des séries en chaque point
    irqdf = transformed_data * 0 + irq
    # Détermination des outliers selon la régle fournie par McCracken et Ng (2016)
    outliers = z > 10 * irqdf
    # Abandon des observations considérées comme outliers
    mapping = transformed_data[outliers == False].dropna(axis=0)
    # Renvoi des séries traitées des outliers
    return mapping

In [10]:
# Lecture du fichier *.txt
data = pd.read_csv('fredmq.txt', sep=',').set_index(keys='Date')
# Récupération des types de transformations dans un dictionnaire
transformations = dict(data.loc['Transform']) 
# Abandon de la ligne des transformations
data.drop(labels='Transform', inplace=True)
# Ajustement de l'indice du DataFrame au format datetime
data.index = pd.to_datetime(arg=data.index)
# Abandon des colonnes ayant un nombre de NaN's >= 30
data.dropna(thresh=len(data) - 30, axis=1, inplace=True)
# Abandon des NaN's restants
data.dropna(axis=0, inplace=True)
# Transformation des séries 
transformed_data = fred_transform(data, transformations)
# Traitement des outliers
mapping = remove_outliers(transformed_data)
# Sélection de la période d'estimation
estimation_sample = mapping.loc[(mapping.index >= pd.to_datetime('1983-01-01')) 
                                & (mapping.index <= pd.to_datetime('2016-12-01'))]
# Sélection de la période de prévision
forecast_sample = mapping.loc[mapping.index >= pd.to_datetime('2017-01-01')]
# Standardisation des données d'estimation
estimation_sample_std = (estimation_sample - estimation_sample.mean(axis=0)) / estimation_sample.std(axis=0)

In [134]:
def principal_components(Y: np.ndarray, r: int) -> tuple:
    """ 
    Extrait les facteurs et les charges d'une série de données standardisée par décomposition en valeur singulière.

    Arguments
    ---------
    Y : np.ndarray
        Matrice (TxN) de données standardisées.
    r : int
        Nombre de facteurs à estimer.

    Renvoie
    ---------
    tuple(np.ndarray, np.ndarray) :
        Matrice (Txr) des facteurs estimés, Matrice (Nxr) des charges factorielles estimées.
    """
    # Décomposition en valeurs singulières de Y'*Y
    U, _, _ = np.linalg.svd(Y.T @ Y)
    # Récupération des r premiers eigenvecteurs
    loadings = U[:, :r]
    # Estimation des facteurs par projection 
    factors = Y @ loadings
    # Renvoi des facteurs et des charges 
    return factors, loadings 

def kalman_filter(Y: np.ndarray, r: int, f0: np.ndarray, 
                  p0: np.ndarray, lbda: np.ndarray, phi: np.ndarray, 
                  sigma_e: np.ndarray, sigma_u: np.ndarray=None) -> tuple:
    """ 
    Applique le filtre de Kalman pour un modèle DFM dans lequel la variable d'etat évolue selon un VAR(p=1).

    Arguments
    ---------
    Y : np.ndarray
        Matrice (TxN) de données standardisées.
    r : int
        Nombre de facteurs à estimer.
    f0 : np.ndarray 
        Vecteur (rx1) des facteurs initiaux.
    p0 : np.ndarray
        Matrice (rxr) de covariance des facteurs initiaux.
    lbda : np.ndarray
        Matrice (Nxr) des charges factorielles.
    phi : np.ndarray
        Matrice (rxr) des paramètres autorégressifs du VAR.
    sigma_e : np.ndarray
        Matrice (NxN) diagonale des variances des composantes idiosyncratiques des N variables.
    sigma_u : np.ndarray
        Matrice (rxr) de covariance des erreurs ut (Si passé comme None alors sigma_u = Ir).

    Renvoie
    ---------
    tuple(np.ndarray, np.ndarray, float) :
        Matrice (Txr) des facteurs filtrés, Vecteur (Tx1) des matrices (NxN) de covariance des facteurs filtrés, log-vraissemblance.
    """
    # Récupération du nombre d'observations et de variables
    T, N = Y.shape
    # Initialisation de la log-vraissemblance
    loglik = 0
    # Initialisation du vecteur contenant les facteurs filtrés
    f_filtered = np.zeros(shape=(T, r))
    # Initialisation du vecteur contenant les matrices de covariance des facteurs filtrés
    p_filtered = np.zeros(shape=(T, r, r))
    # Initialisation du filtre 
    f_prev, p_prev = f0, p0
    # Vérification du type de sigma_u
    if sigma_u is None:
        # Si sigma u est None alors sigma_u = Ir 
        sigma_u = np.eye(r)
    # Application du filtre pour t = 1,...,T
    for t in range(T):

        # Sélection du vecteur (Nx1) des données observables en t
        yt = Y[t, :].reshape((N, 1))
    
        # ---------------------------------
        # Etape n°1 -> prédiction
        # ---------------------------------
        f_pred = phi @ f_prev # Calcul de ft/t-1
        p_pred = phi @ p_prev @ phi.T + sigma_u # Calcul de Pt/t-1
        sigma = lbda @ p_pred @ lbda.T + sigma_e # Calcul de sigma_t/t-1 (matrice de var-cov des innovations)
        # ---------------------------------
        # Etape n°2 -> mise à jour
        # ---------------------------------
        sigma_e_inv = np.linalg.inv(sigma_e) # Inversion de la matrice sigma_e
        sigma_inv = sigma_e_inv - (sigma_e_inv @ lbda) @ np.linalg.inv(np.linalg.pinv(p_pred) + lbda.T @ sigma_e_inv @ lbda) @ (lbda.T @ sigma_e_inv) # Calcul de l'inverse de sigma_t/t-1 via l'identité de Woodburry
        nu = yt - lbda @ f_pred # Calcul de l'innovation
        kt = p_pred @ lbda.T @ sigma_inv # Calcul de la matrice de gain de Kalman
        f_est = f_pred + kt @ nu # Calcul de ft/t
        p_est = p_pred - kt @ lbda @ p_pred # Calcul de Pt/t
        # ---------------------------------

        # Calcul de la log-vraissemblance en t
        loglik += -.5 * (np.log(np.linalg.det(sigma)) + nu.T @ sigma_inv @ nu)
        # Sauvegarde de l'etat filtré
        f_filtered[t] = f_est.flatten() # .flatten() s'assure que f_est soit unidimensionnel
        # Sauvegarde de la matrice de covariance de l'etat filtré
        p_filtered[t] = p_est
        # Mise a jour des estimations précédentes
        f_prev, p_prev = f_est, p_est

    # Calcul final de la log-vraissemblance
    loglik += -.5 * T * N  * np.log(2 * np.pi)
    # Renvoi des etats filtrés, des matrices de covariance et de la log-vraissemblance
    return f_filtered, p_filtered, loglik.item() # .item() car loglik est sous forme ndarray à cause des opérations vectorielles

In [140]:
def tsls(Y: np.ndarray, r: int) -> tuple:
    """
    
    """
    # Récupération du nombre d'observations
    T, _ = Y.shape
    # Estimation des facteurs et charges par analyse en composantes principales 
    pc_factors, pc_loadings = principal_components(Y, r)
    # Estimation des paramètres autorégressifs à l'aide des facteurs extraits
    phi0 = np.linalg.inv(pc_factors[:-1].T @ pc_factors[:-1]) @ (pc_factors[:-1].T @ pc_factors[1:])
    # Calcul des résidus estimés
    epsilon_tilde = Y - pc_factors @ pc_loadings.T
    # Calcul de la matrice de covariance (diagonale) des composants idiosyncratiques
    sigma_e0 = np.diag(np.sum(epsilon_tilde ** 2, axis=0) / T)
    # Calcul des erreurs u
    u_tilde = pc_factors[1:] - pc_factors[:-1] @ phi0.T
    # Calcul de la matrice de covariance des erreurs u
    sigma_u0 = (u_tilde.T @ u_tilde) / T
    # Initialisation du vecteur des facteurs initiaux par les moyennes inconditionnelles des facteurs extraits par PC
    f0 = np.mean(pc_factors, axis=0).reshape((r, 1))
    # Initialisation de la matrice de covariance des facteurs initiale par la covariance inconditionnelle des facteurs extraits par PC
    p0 = np.cov(pc_factors.T).reshape((r, r))
    # Application du filtre de Kalman à l'aide des matrices estimées par PC
    f_filtered, p_filtered, _ = kalman_filter(
        Y=Y,
        r=r,
        f0=f0,
        p0=p0,
        lbda=pc_loadings,
        phi=phi0,
        sigma_e=sigma_e0,
        sigma_u=sigma_u0
    )